In [1]:
import nivapy3 as nivapy
import nope
import useful_rid_code as rid
import pandas as pd

# Generate overall summary table - 2018

Table 3 of the 2015 report summarises all monitored and modelled loads for the whole of Norway, and for the four OSPAR regions. This notebook reads the output from the processing of the observed data (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_working_2016-17.ipynb)) and the modelling using NOPE (see [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/loads_unmonitored_regions_2016.ipynb)) and generates a Word version of Table 3. The code is based on the exploratory work [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/word_data_tables.ipynb).

## 1. Summarise monitoring data

In [2]:
# Read station data
in_xlsx = r'../../../Data/RID_Sites_List_2017-2020.xlsx'
stn_df = pd.read_excel(in_xlsx, sheet_name='RID_All')

# Get just cols of interest and drop duplicates 
# (some sites are in the same regine)
stn_df = stn_df.drop_duplicates(subset=['ospar_region', 'nve_vassdrag_nr'])

# Get catch IDs with calib data
calib_nds = set(stn_df['nve_vassdrag_nr'].values)

# Build network
in_path = r'../../../NOPE/NOPE_Annual_Inputs/nope_input_data_1990.csv'
g, nd_list = nope.build_calib_network(in_path, calib_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df['nve_vassdrag_nr'].isin(ds_nds)]

**Note:** Remember to update the year below.

In [3]:
# Year of interest
year = 2019

In [4]:
# Read data
in_csv = r'../../../Results/Loads_CSVs/loads_and_flows_all_sites_%s.csv' % year
mon_df = pd.read_csv(in_csv)

# Get just the downstream catchments
#mon_df = mon_df[mon_df['station_id'].isin(stn_df['station_id'].values)]

# Group by OSPAR region
mon_df1 = mon_df.groupby(['ospar_region', 'new_rid_group']).sum()

# Totals for Norway
mon_df2 = mon_df.groupby('new_rid_group').sum().reset_index()
mon_df2['ospar_region'] = 'NORWAY'
mon_df2.set_index(['ospar_region', 'new_rid_group'], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split('_')[1] != 'Est']
mon_df = mon_df[cols]
del mon_df['station_id']

# Rename cols to match template
mon_df['Flow rate_1000m3/day'] = mon_df['mean_q_1000m3/day']
del mon_df['mean_q_1000m3/day']

# Units are correct, so remove
mon_df.columns = [i.split('_')[0] for i in mon_df.columns]

mon_df.round(0)

Ag    As   Cd    Cr     Cu     Hg  NH4-N  \
ospar_region        new_rid_group                                              
LOFOTEN-BARENTS SEA rid_135        0.0   1.0  0.0   1.0    8.0    7.0  103.0   
                    rid_20         0.0   2.0  0.0   7.0   63.0    8.0  260.0   
NORTH SEA           rid_135        0.0   3.0  0.0   3.0   18.0   25.0  278.0   
                    rid_20         0.0   1.0  0.0   0.0    4.0    5.0   24.0   
NORWEGIAN SEA2      rid_135        0.0   4.0  0.0   9.0   30.0   29.0  329.0   
                    rid_20         0.0   1.0  0.0   2.0   17.0   11.0   27.0   
SKAGERAK            rid_135        0.0   4.0  0.0   2.0   11.0   15.0  241.0   
                    rid_20         0.0   8.0  1.0   9.0   60.0  113.0  625.0   
NORWAY              rid_135        0.0  11.0  1.0  16.0   67.0   76.0  951.0   
                    rid_20         0.0  12.0  1.0  18.0  144.0  136.0  935.0   

                                     NO3-N     Ni  PO4-P    Pb       SPM  \
ospar_region        new_rid_group                                          
LOFOTEN-BARENTS SEA rid_135          355.0    6.0   19.0   0.0   17888.0   
                    rid_20           919.0  100.0   64.0   2.0   79803.0   
NORTH SEA           rid_135         5599.0    9.0   66.0   6.0   59513.0   
                    rid_20           917.0    2.0   12.0   1.0    6657.0   
NORWEGIAN SEA2      rid_135         2802.0   22.0   94.0   3.0  123895.0   
                    rid_20          1334.0    6.0   19.0   0.0   14628.0   
SKAGERAK            rid_135         3515.0    7.0   40.0   5.0   34987.0   
                    rid_20         17031.0   30.0  288.0  10.0  281275.0   
NORWAY              rid_135        12271.0   44.0  220.0  14.0  236283.0   
                    rid_20         20201.0  138.0  383.0  12.0  382364.0   

                                       SiO2       TOC     TOTN   TOTP     Zn  \
ospar_region        new_rid_group                                              
LOFOTEN-BARENTS SEA rid_135         33534.0   31425.0   1599.0   34.0    9.0   
                    rid_20         109330.0   66012.0   3289.0  193.0   60.0   
NORTH SEA           rid_135         53678.0   60667.0   9833.0  189.0   68.0   
                    rid_20           6143.0    9307.0   1501.0   37.0   13.0   
NORWEGIAN SEA2      rid_135         79001.0   85079.0   8135.0  216.0   70.0   
                    rid_20          27196.0   25019.0   2296.0   46.0   32.0   
SKAGERAK            rid_135         36324.0   76241.0   7079.0  154.0   68.0   
                    rid_20         167748.0  219018.0  26010.0  662.0  221.0   
NORWAY              rid_135        202536.0  253412.0  26645.0  594.0  214.0   
                    rid_20         310417.0  319356.0  33097.0  937.0  327.0   

                                   Flow rate  
ospar_region        new_rid_group             
LOFOTEN-BARENTS SEA rid_135          41214.0  
                    rid_20           69526.0  
NORTH SEA           rid_135         120655.0  
                    rid_20           13501.0  
NORWEGIAN SEA2      rid_135         123180.0  
                    rid_20           34611.0  
SKAGERAK            rid_135          41994.0  
                    rid_20          146439.0  
NORWAY              rid_135         327043.0  
                    rid_20          264077.0

## 2. Data for unmonitored areas

**Note:** Remember to update the year below.

In [5]:
# Read data
in_csv = r'../../../Results/Unmon_loads/unmon_loads_%s.csv' % year
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace('RENSEANLEGG', 'sew') for i in umon_df.columns]
umon_df.columns = [i.replace('INDUSTRI', 'ind') for i in umon_df.columns]
umon_df.columns = [i.replace('_tonn', '') for i in umon_df.columns]
umon_df.columns = [i.replace('AQUAKULTUR', 'fish') for i in umon_df.columns]

# Convert Hg to kgs
umon_df['sew_Hg'] = umon_df['sew_Hg']*1000
umon_df['ind_Hg'] = umon_df['ind_Hg']*1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,425297.0,13310.0,1104.0,2476.0,231.0,63365.0,10925.0,36780.0,725.0,662.0,...,15640.0,0.0,0.0,0.0,6.0,15.0,2.0,0.0,3.0,1217.0
LOFOTEN-BARENTS SEA,127196.0,1543.0,180.0,27.0,2.0,15664.0,2712.0,5953.0,111.0,108.0,...,2506.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,302.0
NORTH SEA,134234.0,3534.0,441.0,470.0,83.0,20361.0,3501.0,14173.0,217.0,265.0,...,6169.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,389.0
NORWEGIAN SEA2,150979.0,2795.0,367.0,1051.0,105.0,27283.0,4703.0,13422.0,302.0,220.0,...,4565.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,525.0
SKAGERAK,12888.0,5438.0,116.0,927.0,41.0,56.0,10.0,3232.0,96.0,70.0,...,2400.0,0.0,0.0,0.0,4.0,10.0,1.0,0.0,1.0,1.0


## 3. Create table

The cell below uses the 3-fold division of sites (`'rid_11'` etc.) from the old progreamme structure. To use this, copy the old Word template and make sure that the aggregation in Section 1 is done by `'old_rid_group'`.

In [6]:
## Create table 3 (old programme structure)
#in_docx = f'../../../Results/Word_Tables/{year + 1}Analysis_{year}Data/Table_3_{year}.docx'
#
#rid.write_word_overall_table(mon_df, umon_df, in_docx)

The cell below uses the 2-fold division of sites (`'rid_20'` and `'rid_135'`) for the 2017-20 progreamme structure. To use this, copy the new Word template and make sure that the aggregation in Section 1 is done by `'new_rid_group'`.

In [8]:
# Create table 3
in_docx = f'../../../Results/Word_Tables/{year + 1}Analysis_{year}Data/Table_3_{year}_new.docx'

rid.write_word_overall_table_2017_20(mon_df, umon_df, in_docx)

Finished.
